# Selecting Data for Modeling

Your dataset had too many variables to wrap your head around, or even to print out nicely. How can you pare down this overwhelming amount of data to something you can understand?

We'll start by picking a few variables using our intuition. Later courses will show you statistical techniques to automatically prioritize variables.

To choose variables/columns, we'll need to see a list of all columns in the dataset. That is done with the columns property of the DataFrame (the bottom line of code below).

In [5]:
# import libraries
import pandas as pd

In [1]:
# load and describe data
melbourne_data = pd.read_csv('../../dataset/melb_data.zip')
melbourne_data.describe()

# dropna drops missing values (think of na as "not available")
melbourne_data = melbourne_data.dropna(axis=0)

# Selecting The Prediction Target
y = melbourne_data.Price

NameError: name 'pd' is not defined

## Selecting Features

In [9]:
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
# by convention, this data is called X
X = melbourne_data[melbourne_features]
X.describe()

## Building Your Model

You will use the scikit-learn library to create your models. When coding, this library is written as sklearn, as you will see in the sample code. Scikit-learn is easily the most popular library for modeling the types of data typically stored in DataFrames.

The steps to building and using a model are:

Define: What type of model will it be? A decision tree? Some other type of model? Some other parameters of the model type are specified too.
Fit: Capture patterns from provided data. This is the heart of modeling.
Predict: Just what it sounds like
Evaluate: Determine how accurate the model's predictions are.
Here is an example of defining a decision tree model with scikit-learn and fitting it with the features and target variable

In [11]:
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
melbourne_model = DecisionTreeRegressor(random_state=1)

# Fit model
melbourne_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

Many machine learning models allow some randomness in model training. Specifying a number for random_state ensures you get the same results in each run. This is considered a good practice. You use any number, and model quality won't depend meaningfully on exactly what value you choose.

We now have a fitted model that we can use to make predictions.

In practice, you'll want to make predictions for new houses coming on the market rather than the houses we already have prices for. But we'll make predictions for the first few rows of the training data to see how the predict function works.

In [12]:
print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(melbourne_model.predict(X.head()))

Making predictions for the following 5 houses:
   Rooms  Bathroom  Landsize  Lattitude  Longtitude
1      2       1.0     156.0   -37.8079    144.9934
2      3       2.0     134.0   -37.8093    144.9944
4      4       1.0     120.0   -37.8072    144.9941
6      3       2.0     245.0   -37.8024    144.9993
7      2       1.0     256.0   -37.8060    144.9954
The predictions are
[1035000. 1465000. 1600000. 1876000. 1636000.]


## Model Validation

You'll want to evaluate almost every model you ever build. In most (though not all) applications, the relevant measure of model quality is predictive accuracy. In other words, will the model's predictions be close to what actually happens.

So, if a house cost 150,000 and you predicted it would cost 100,000 the error is 50,000.

In [13]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
MAE = mean_absolute_error(y, predicted_home_prices)

print(MAE)

1115.7467183128902


The measure we just computed can be called an "in-sample" score. We used a single "sample" of houses for both building the model and evaluating it. Here's why this is bad.

In [14]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
X, X_test, y, y_test = train_test_split(X, y, random_state = 0)

# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(X, y)

# get predicted prices on validation data

val_predictions = melbourne_model.predict(X_test)

# get MSE (mean absolute error)
MSE = mean_absolute_error(y_test, val_predictions)
print(MSE)

276173.46417043253


## Underfitting and Overfitting

There are a few alternatives for controlling the tree depth, and many allow for some routes through the tree to have greater depth than other routes. But the max_leaf_nodes argument provides a very sensible way to control overfitting vs underfitting. The more leaves we allow the model to make, the more we move from the underfitting area in the above graph to the overfitting area.

We can use a utility function to help compare MAE scores from different values for max_leaf_nodes:

In [15]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, test_X, train_y, test_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    prediction_val = model.predict(test_X)
    mae = mean_absolute_error(test_y, prediction_val)
    return mae


In [17]:
# compare MAE with differing values of max_leaf_nodes
optimal_node = 0
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, X, X_test, y, y_test)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))
    if optimal_node == 0 or my_mae < optimal_node:
        optimal_node = my_mae
        optimal_node_value = max_leaf_nodes
print("Optimal node: %d" % optimal_node_value)

Max leaf nodes: 5  		 Mean Absolute Error:  385696
Max leaf nodes: 50  		 Mean Absolute Error:  279794
Max leaf nodes: 500  		 Mean Absolute Error:  261718
Max leaf nodes: 5000  		 Mean Absolute Error:  271320
Optimal node: 500


- Overfitting: capturing spurious patterns that won't recur in the future, leading to less accurate predictions
- Underfitting: failing to capture relevant patterns, again leading to less accurate predictions.

In [18]:
# Fit model with optimal leaf nodes

model = DecisionTreeRegressor(max_leaf_nodes=optimal_node_value, random_state=0)

model.fit(X, y)

# get predicted prices on validation data
val_predictions = model.predict(X_test)

# get MSE (mean absolute error)
MSE = mean_absolute_error(y_test, val_predictions)
print(MSE)

261718.1134423186
